In [2]:
# Load YAML configuration file
import yaml

with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Load dataset with pandas
import pandas as pd
df = pd.read_csv(config['input_data']['file'])  # Load dataset based on YAML path
df.head()  # Show the first 5 rows of the dataset

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type,genres,platform
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",90,min,['Documentaries'],Netflix
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2,season,"['International TV Shows', 'TV Dramas', 'TV My...",Netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,season,"['Crime TV Shows', 'International TV Shows', '...",Netflix
3,s4,TV Show,Jailbirds New Orleans,Unknown,Unknown,Unknown,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1,season,"['Docuseries', 'Reality TV']",Netflix
4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2,season,"['International TV Shows', 'Romantic TV Shows'...",Netflix


In [ ]:
# Explore structure and metadata
df.shape         # Check number of rows and columns
df.columns       # List all column names
df.info()        # Get data types and non-null counts per column

In [ ]:
# Check missing values in descending order
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Number of unique values per column
df.nunique().sort_values()

# Distribution of key categorical columns
df['type'].value_counts(normalize=True)  # Distribution of 'Movie' vs 'TV Show'
df['country'].value_counts().head(10)    # Top 10 countries
df['rating'].value_counts().head(10)     # Most common ratings

In [ ]:
# View a sample of the genres column
df['listed_in'].sample(10)

In [ ]:
# Check unique countries
df['country'].unique()[:20]

In [ ]:
# Split 'duration' column into duration value and type (e.g., "90 min" → 90 + 'min')
df[['duration_int', 'duration_type']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')
df['duration_int'] = pd.to_numeric(df['duration_int'], errors='coerce')  # Convert to numeric
df[['duration', 'duration_int', 'duration_type']].head()  # Preview new columns

In [ ]:
# Convert 'date_added' column to datetime format
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce') # Convert to datetime
df['date_added'].dtype  # Confirm conversion

In [ ]:
# Split genres into lists (for multi-label classification)
df['genres'] = df['listed_in'].str.split(', ') # Split genre strings into lists
df[['listed_in', 'genres']].sample(5)  # Check results

In [ ]:
# Frequency of duration types (e.g., 'min', 'Season')
df['duration_type'].value_counts() # Check types of duration (minutes or seasons)

# Standardize plural form to singular ('Seasons' → 'Season')
df['duration_type'] = df['duration_type'].replace('Seasons', 'Season') # Normalize to singular
df['duration_type'].value_counts()  # Confirm correction

In [ ]:
# Check for missing values in 'duration_int'
df['duration_int'].isnull().sum()  # Check how many null durations remain
# Keep only rows where 'duration_int' is not null
df = df[df['duration_int'].notna()]  # Filter out rows without valid duration
# Inspect rows where duration was missing
df[df['duration_int'].isna()][['duration']]


In [ ]:
# Remove duplicated rows based on 'show_id', 'title', and 'release_year'
df = df.drop_duplicates(subset=['show_id', 'title', 'release_year'])  # Remove duplicates
print("Number of rows after removing duplicates:", df.shape[0])  # Check number of rows after deduplication

In [ ]:
# Confirm that duplicates are removed
duplicates = df.duplicated(subset=['show_id', 'title', 'release_year']).sum()  # Count remaining duplicates
print("Remaining duplicates:", duplicates) 

In [ ]:
# Check missing values again per column (after cleaning)
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Fill missing values with appropriate defaults
# 1. Replace missing 'director' with "Unknown"
df['director'] = df['director'].fillna("Unknown")
# 2. Replace missing 'cast' with "Unknown"
df['cast'] = df['cast'].fillna("Unknown")
# 3. Replace missing 'country' with "Unknown"
df['country'] = df['country'].fillna("Unknown")
# 4. Drop rows where 'date_added' is missing
df = df[df['date_added'].notna()]
# 5. Replace missing 'rating' with "Not Rated"
df['rating'] = df['rating'].fillna("Not Rated")

In [ ]:
# Confirm again that no missing values remain after filling
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Define a function to extract the first country if there are multiple listed
def extract_first_country(country):
    if isinstance(country, str):
        return country.split(',')[0]  # Keep only the first country
    return country

# Apply the function to create a cleaned 'country' column
df['country'] = df['country'].apply(extract_first_country)

In [ ]:
# Quick check of distributions on key columns
print(df['duration_type'].value_counts())  # Check distribution of 'duration_type'
print(df['type'].value_counts())           # Check distribution of 'type'
print(df['rating'].value_counts())         # Check distribution of 'rating'
print(df['country'].value_counts().head(20))  # Check top 20 most common countries

In [ ]:
# Normalize inconsistent 'rating' labels into unified categories
df['rating'] = df['rating'].replace({
    'NR': 'Unrated',
    'UR': 'Unrated',
    'Not Rated': 'Unrated',
    'UNRATED': 'Unrated',
    'TV-Y7-FV': 'TV-Y7'  # Merge variant of TV-Y7
})

In [ ]:
# Standardize 'duration_type' to lowercase for consistency
df['duration_type'] = df['duration_type'].str.lower()

In [ ]:
# Check the format and range of 'date_added' column
print("Date format:", df['date_added'].dtype)  # Confirm 'date_added' is datetime
print("Earliest date:", df['date_added'].min())  # Check earliest date
print("Latest date:", df['date_added'].max())    # Check latest date
df['date_added'].sample(10, random_state=1)     # Preview 10 random samples

In [ ]:
# Calculate the average duration by 'duration_type' (e.g., average movie length)
avg_duration = df.groupby('duration_type')['duration_int'].mean().reset_index()
print("Average duration per type:")
print(avg_duration)

In [ ]:
# Count number of titles per country (using the cleaned 'country' column)
country_counts = df['country'].value_counts().reset_index()
country_counts.columns = ['country', 'count']
print("\nNumber of titles by country:")
print(country_counts.head(10))

In [ ]:
# Explode the 'genres' column and count most common genres
df_exploded = df.explode('genres')  # Separate multiple genres into individual rows
genre_counts = df_exploded['genres'].value_counts().reset_index()  # Count occurrences of each genre
genre_counts.columns = ['genre', 'count']  # Rename columns for clarity
print("\nMost common genres:")
print(genre_counts.head(10))  # Show top 10 genres

In [ ]:
# Count number of titles by release year
titles_per_year = df['release_year'].value_counts().sort_index().reset_index()
titles_per_year.columns = ['release_year', 'count']
print("\n🔹 Titles by release year:")
print(titles_per_year.tail(10))  # Show most recent years

In [ ]:
#Export cleaned DataFrame to CSV for SQL import or future use
df['platform'] = 'Netflix'
df.to_csv('../data/clean/cleaned_netflix_data_with_platform.csv', index=False) #Save


In [ ]:
print(df.columns)
print(df[['show_id', 'title', 'platform']].head())

In [ ]:
import os
print("File updated successfully:", os.path.getmtime('../data/clean/cleaned_netflix_data_with_platform.csv'))
